# Consume messages from an Apache Kafka partitioned topic

In this notebook you will learn how to consume messages to an Apache Kafka topic with two partitions (created with the [dedicated notebook](3-produce-partitioned-topic.ipynb))

![Consume messages from an Apache Kafka Topic](../img/consume.png)

---

## Prerequisites

To start the tutorial you need to:

* Download the Aiven for Apache Kafka SSL certificates as mentioned in the [README instructions](../README.md#Download-the-required-SSL-certificates)
* Retrieve the Aiven for Apache Kafka hostname and port, from the [Aiven Console](https://console.aiven.io/) in the Aiven for Apache Kafka service overview
* Substitute the Apache Kafka hostname and port in the parameters below

In [ ]:
# Replace the following two placeholders with Aiven for Apache Kafka service hostname and port

hostname="<MYKAFKAHOST>"
port="<MYKAFKAPORT>"

---

### Install the required libraries

The following installs the [kafka-python library](https://kafka-python.readthedocs.io/en/master/) we'll use for the tutorial

In [ ]:
!pip install kafka-python

---

## Create an Apache Kafka Consumer

The next step is to configure the Kafka Consumer.

In [ ]:
import json

from confluent_kafka import DeserializingConsumer

def json_serializer(msg, s_obj):
    return json.loads(msg.decode('ascii'))

conf = {
    'bootstrap.servers': hostname+":"+port,
    'client.id': 'myclient',
    'group.id': 'pizzaioli',
    'security.protocol': 'SSL',
    'ssl.ca.location': '../sslcerts/ca.pem',
    'ssl.certificate.location': '../sslcerts/service.cert',
    'ssl.key.location': '../sslcerts/service.key', 
    'value.deserializer': json_serializer,
    'key.deserializer': json_serializer
    }

consumer = DeserializingConsumer(conf)

---
## Consume messages to the `pizzaPartitioned` topic, partition 0

In [ ]:
from confluent_kafka import TopicPartition

running = True

try:
    consumer.assign(
    [
        TopicPartition(
        "pizzaPartitioned", 
        0)
    ])

    while running:
        msg = consumer.poll(timeout=1.0)
        if msg is None: continue

        if msg.error():
            if msg.error().code() == KafkaError._PARTITION_EOF:
                # End of partition event
                sys.stderr.write('%% %s [%d] reached end at offset %d\n' %
                                 (msg.topic(), msg.partition(), msg.offset()))
            elif msg.error():
                raise KafkaException(msg.error())
        else:
            print ("%d:%d: k=%s v=%s" % (
                msg.partition(),
                msg.offset(),
                msg.key(),
                msg.value()))
except KeyboardInterrupt:
    print ("Shutdown requested...exiting")
finally:
    # Close down consumer to commit final offsets.
    consumer.close() 